In [1]:
%pylab inline
from scipy.interpolate import interpn
from constant import * 
from multiprocessing import Pool
from functools import partial
import warnings
warnings.filterwarnings("ignore")
np.printoptions(precision=2)

Populating the interactive namespace from numpy and matplotlib


### Value iteration of renting a house

$$V_t(w_t, s_t) = max_{\{c_t, h_t, b_t, k_t\}}\{ u(c_t, h_t) + \beta E[P_t V_{t+1}(w_{t+1}, s_{t+1}) + (1-P_t) u_B(w_{t+1})]\}$$
Subject to:    
$$y_t + w_t = c_t + P_t^r h_t + b_t + k_t$$
$$ w_{t+1} = (1+r_t^f) b_t + (1+r_{t+1}^m) k_t $$    

Shape of the state: $$(100  \times 27)$$
Shape of the actions: $$(20 \times 20 \times 20)$$

In [2]:
nX = Xs.shape[0]
Xs.shape

(2700, 2)

In [3]:
nA = As.shape[0]
As.shape

(8000, 3)

In [4]:
X = np.repeat(Xs, nA, axis = 0)
A = np.tile(As, (nX,1))

In [5]:
A.shape

(21600000, 3)

In [6]:
X.shape

(21600000, 2)

In [7]:
#Define the utility function
def u(c):
    return (np.float_power(c, 1-gamma) - 1)/(1 - gamma)

#Define the bequeath function, which is a function of wealth
def uB(tb):
    return B*u(tb)

#Reward function for renting
def u_rent(a):
    '''
    Input:
        action a: c, b, k, h = a
    Output:
        reward value: the length of return should be equal to the length of a
    '''
    c = a[:,0]
    h = a[:,3]
    C = np.float_power(c, alpha) * np.float_power(h, 1-alpha)
    return u(C)

def transition_to_rent(x,a,t):
    '''
        imput: a is np array constains all possible actions 
        output: from x = w_t to x = w_{t+1}
    '''
    s = x[:,1].astype(int)
    # actions taken 
    b = a[:,1]
    k = a[:,2]
    w_next = ((1+r_b[s])*b + np.outer(k,(1+r_k)).T).T
    w_next = w_next.flatten()
    s_next = np.tile(range(nS),nA*nX)
    prob = Ps[s].flatten()
    return w_next, s_next, prob

class Approxy(object):
    def __init__(self, V, pts, t):
        self.V = V 
        self.pts = pts
        self.t = t
    def predict(self, xx):
        # x = [w, s]
        pvalues = np.zeros(xx.shape[0])
        for s in range(nS): 
            index = (xx[:,1] == s)
            pvalues[index]=interpn(self.pts, self.V[:,s], xx[index][:,0], bounds_error = False, fill_value = None)
        return pvalues

In [8]:
t = 10
yat = detEarning[t]
budget1 = yat + X[:,0]
h = budget1*A[:,0]/pr
budget2 = budget1*(1-A[:,0])
c = budget2*A[:,1]
budget3 = budget2*(1-A[:,1])
k = budget3*A[:,2]
b = budget3*(1-A[:,2])
actions = np.column_stack((c,b,k,h))
w_next, s_next, prob = transition_to_rent(X,actions,0)

In [9]:
w_next.shape

(583200000,)

In [10]:
s_next.shape

(583200000,)

In [ ]:
prob.s

In [15]:
t = 10
yat = detEarning[t]
budget1 = yat + X[:,0]
h = budget1*A[:,0]/pr
budget2 = budget1*(1-A[:,0])
c = budget2*A[:,1]
budget3 = budget2*(1-A[:,1])
k = budget3*A[:,2]
b = budget3*(1-A[:,2])
actions = np.column_stack((c,b,k,h))
x = X
a = actions
s = x[:,1].astype(int)
# actions taken 
b = a[:,1]
k = a[:,2]
w_next = ((1+r_b[s])*b + np.outer(k,(1+r_k)).T).T
w_next = w_next.flatten()
s_next = np.tile(range(nS),nA*nX)
prob = Ps[s].flatten()

In [ ]:
%%time
for t in range(T_max-1,T_min, -1):
    print(t)
    # x = [w,s], a = [c,b,k,h]
    yat = detEarning[t]
    budget1 = yat + X[:,0]
    h = budget1*A[:,0]/pr
    budget2 = budget1*(1-A[:,0])
    c = budget2*A[:,1]
    budget3 = budget2*(1-A[:,1])
    k = budget3*A[:,2]
    b = budget3*(1-A[:,2])
    actions = np.column_stack((c,b,k,h))
    
    u_rent(actions) = 
    
    if t == T_max - 1:
        f = partial(V, t = t, NN = None)
        results = np.array(pool.map(f, xs))
    else:
        approx = Approxy(Vgrid[:,:,t+1],pointsRent, t+1)
        f = partial(V, t = t, NN = approx)
        results = np.array(pool.map(f, xs))
    Vgrid[:,:,t] = results[:,0].reshape(dim)
    cgrid[:,:,t] = np.array([r[0] for r in results[:,1]]).reshape(dim)
    bgrid[:,:,t] = np.array([r[1] for r in results[:,1]]).reshape(dim)
    kgrid[:,:,t] = np.array([r[2] for r in results[:,1]]).reshape(dim)
    hgrid[:,:,t] = np.array([r[3] for r in results[:,1]]).reshape(dim)